##**Survey**

This notebook aims to process and analyze the survey datasets. The survey results are divided in 8 files with 3 companies and 3 annotators. The format of Qualtrics data could be confusing. In each dataset, the rows are annotators and the questions are columns. The columns questions have been formatted with the company ID, the number of the task, the risk ID and type of the answer.



*   Task 1 column name from Qualtrics: ***Q[company_id].1*** from multiple choice and for open text responses ***Q[company_id].1_[risk_id]_TEXT***
*   Task 2 column name from Qualtrics: ***Q[company_id].2_[risk_id]*** from the Yes/No selection and for open text responses ***Q[company_id].2_[risk_id]_TEXT.***


The final output is a dataset called *2_companies_survey.csv* with the answers in one file and stored appropiately for comparison with the LLMs.


In [ ]:
pip install krippendorff openpyxl

**Libraries**

In [ ]:
import pandas as pd
import numpy as np
import krippendorff

**Companies database**

The database output from the first notebook with the usecases and the summarized description. A random sample of companies will be extracted to use on the survey.

In [ ]:
companies = pd.read_csv('1_companies_df.csv')
companies['Company ID'] = companies['ID'].astype(str)
companies_survey = companies.sample(n=24, random_state=58) #the random sample used on the survey
#companies_survey.to_csv("subset_missions.csv", index=False)

**Survey dataset**

The survey is in 8 files, each containing three annotators in rows and the questions for three companies in columns.
I merged the 8 files into one survey dataset.

In [ ]:
import os
import re

# List of the survey files
survey_files = ['Survey/Survey_1.csv', 'Survey/Survey_2.csv', 'Survey/Survey_3.csv', 'Survey/Survey_4.csv',
                'Survey/Survey_5.csv', 'Survey/Survey_6.csv', 'Survey/Survey_7.csv', 'Survey/Survey_8.csv']
dfs = []

# Read each file, add survey_id, and append to the dfs list
for file in survey_files:
    if os.path.exists(file):
        try:
            # Read the CSV, skipping the first two rows as these rows dont contain the name of the columns
            #these rows contain the question text as appeared on Qualtrics
            df = pd.read_csv(file, low_memory=False)
            df = df.drop(index=[0, 1])

            # Extract survey ID from the filename and append it as a new column
            match = re.search(r'Survey_(\d+)\.csv$', file)
            if match:
                survey_id = match.group(1)
                df['survey_id'] = survey_id

            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    else:
        print(f"File not found: {file}")


# Concatenate all files
if dfs:
    survey = pd.concat(dfs, ignore_index=True)
    survey = survey.dropna(subset=['Dem_1'])
    display(survey.head())
    print(f"Concatenated survey data shape: {survey.shape}")

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q400.2_3,Q400.2_3_TEXT,Q400.2_4,Q400.2_4_TEXT,Q400.2_5,Q400.2_5_TEXT,Q400.2_6,Q400.2_6_TEXT,Q400.2_7,Q400.2_7_TEXT
0,01/08/2025 01:21,01/08/2025 01:56,IP Address,103.104.20.152,100,2109,TRUE,01/08/2025 01:56,R_2EaH3PsQ7C7nyes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/08/2025 01:48,01/08/2025 02:42,IP Address,90.198.141.113,100,3182,TRUE,01/08/2025 02:42,R_8TvDTYDaFluZ0wV,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01/08/2025 04:33,01/08/2025 05:20,IP Address,90.242.153.197,100,2818,TRUE,01/08/2025 05:20,R_8lyO3A2TvbFIHgA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-08-05 09:58:19,2025-08-05 10:14:28,IP Address,51.19.204.78,100,969,True,2025-08-05 10:14:29,R_8p4Rpk2AmTDIarL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-08-05 14:03:53,2025-08-05 14:36:36,IP Address,80.7.201.61,100,1963,True,2025-08-05 14:36:37,R_2dgs1UnWDkB9E0i,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Concatenated survey data shape: (24, 553)


**Prolific demographic information**

The following file was downloaded from Prolific. It contains the demographic information of participants that were approved. The dataset has the variables used for filtering participants (Highest education, use of AI, work function) and demographic information about Sex and Age and the Prolific Id.

In [ ]:
demographics = pd.read_csv('Survey/Prolific_demographic_information.csv')
demographics.head()

,Participant id,Weekly ai (artificial intelligence) use,Highest education level completed,Work function,Age,Sex
0,684f05593c8301d7db6ecd83,Multiple times every day,Doctorate degree (PhD/other),Engineering (e.g. software),30,Female
1,6759faea38faca8e359a5f99,Multiple times every day,Graduate degree (MA/MSc/MPhil/other),Data Analysis,19,Female
2,59dccd61e75b450001a689b7,2-6 times a week,Graduate degree (MA/MSc/MPhil/other),Engineering (e.g. software),57,Male
3,67cec27daaf29e150e2440c0,Multiple times every day,Graduate degree (MA/MSc/MPhil/other),IT / Information Networking / Information Secu...,23,Male
4,68123a73ba06c472ccd5edb8,Multiple times every day,Graduate degree (MA/MSc/MPhil/other),IT / Information Networking / Information Secu...,45,Female


In [ ]:
#To merge the full survey dataset with the demographics
survey= survey.merge(demographics, left_on='Dem_1', right_on='Participant id', how='left')
survey.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q400.2_6,Q400.2_6_TEXT,Q400.2_7,Q400.2_7_TEXT,Participant id,Weekly ai (artificial intelligence) use,Highest education level completed,Work function,Age,Sex
0,01/08/2025 01:21,01/08/2025 01:56,IP Address,103.104.20.152,100,2109,TRUE,01/08/2025 01:56,R_2EaH3PsQ7C7nyes,NaN,...,NaN,NaN,NaN,NaN,684f05593c8301d7db6ecd83,Multiple times every day,Doctorate degree (PhD/other),Engineering (e.g. software),30,Female
1,01/08/2025 01:48,01/08/2025 02:42,IP Address,90.198.141.113,100,3182,TRUE,01/08/2025 02:42,R_8TvDTYDaFluZ0wV,NaN,...,NaN,NaN,NaN,NaN,6759faea38faca8e359a5f99,Multiple times every day,Graduate degree (MA/MSc/MPhil/other),Data Analysis,19,Female
2,01/08/2025 04:33,01/08/2025 05:20,IP Address,90.242.153.197,100,2818,TRUE,01/08/2025 05:20,R_8lyO3A2TvbFIHgA,NaN,...,NaN,NaN,NaN,NaN,59dccd61e75b450001a689b7,2-6 times a week,Graduate degree (MA/MSc/MPhil/other),Engineering (e.g. software),57,Male
3,2025-08-05 09:58:19,2025-08-05 10:14:28,IP Address,51.19.204.78,100,969,True,2025-08-05 10:14:29,R_8p4Rpk2AmTDIarL,NaN,...,NaN,NaN,NaN,NaN,672de648af50014c67c693db,Every day,Graduate degree (MA/MSc/MPhil/other),Engineering (e.g. software),32,Female
4,2025-08-05 14:03:53,2025-08-05 14:36:36,IP Address,80.7.201.61,100,1963,True,2025-08-05 14:36:37,R_2dgs1UnWDkB9E0i,NaN,...,NaN,NaN,NaN,NaN,668e9165d55b3b68f850838f,Multiple times every day,Doctorate degree (PhD/other),Engineering (e.g. software),36,Male


I generated count tables based on the variables extracted from Prolific.

In [ ]:
# Count participants by Sex
sex_counts = survey['Sex'].value_counts().reset_index()
sex_counts.columns = ['Sex', 'Participant Count']
display(sex_counts)

# Count participants by Work function
work_function_counts = survey['Work function'].value_counts().reset_index()
work_function_counts.columns = ['Work function', 'Participant Count']
display(work_function_counts)

# Count participants by Highest education level completed
education_counts = survey['Highest education level completed'].value_counts().reset_index()
education_counts.columns = ['Highest education level completed', 'Participant Count']
display(education_counts)

# Categorize Age into categories
bins = range(18, survey['Age'].max() + 10, 10) # Start from 18, go up to max age + 10 in steps of 10
labels = [f'{i} to {i+9}' for i in bins[:-1]] # Its categorized like '18 to 27', '28 to 37', etc.
survey['Age_Category'] = pd.cut(survey['Age'], bins=bins, labels=labels, right=False)

# Count participants by Age Category
age_category_counts = survey['Age_Category'].value_counts().reset_index()
age_category_counts.columns = ['Age Category', 'Participant Count']
age_category_counts = age_category_counts.sort_values('Age Category') # Sort by age category for better readability
display(age_category_counts)

,Sex,Participant Count
0,Male,17
1,Female,7


,Work function,Participant Count
0,Engineering (e.g. software),9
1,IT / Information Networking / Information Secu...,8
2,Data Analysis,6
3,Research,1


,Highest education level completed,Participant Count
0,Graduate degree (MA/MSc/MPhil/other),18
1,Doctorate degree (PhD/other),6


,Age Category,Participant Count
1,18 to 27,6
0,28 to 37,11
2,38 to 47,3
3,48 to 57,3
4,58 to 67,1


**Risk taxonomy**

The risk domains list is going to be used for mapping.

In [ ]:
mit_risks = pd.read_excel('Missions.xlsx', sheet_name="MIT_domain")
risk_categories = mit_risks['Domain'].dropna().str.strip().tolist()
risk_categories #for mapping

['Discrimination & Toxicity',
 'Misinformation',
 'Privacy & Security',
 'Malicious actors',
 'Human- Computer Interaction',
 'Socioeconomic & Environmental',
 'AI system safety, failures, & limitations']

In [ ]:
# Dynamically find all column names related to Task 1 questions (pattern Q[company_id].1)
import re

company_questions_task1 = [col for col in survey.columns if re.match(r'Q\d+\.1$', col)]

# Extract the ids of the companies from Task 1 columns
company_ids_task1 = sorted(list(set([re.match(r'Q(\d+)\.1$', col).group(1) for col in company_questions_task1])))

print(f"Task 1 company question columns found: {company_questions_task1}")
print(f"Unique company IDs found for Task 1: {company_ids_task1}")

Task 1 company question columns found: ['Q1.1', 'Q290.1', 'Q138.1', 'Q201.1', 'Q104.1', 'Q183.1', 'Q205.1', 'Q425.1', 'Q78.1', 'Q211.1', 'Q20.1', 'Q287.1', 'Q288.1', 'Q426.1', 'Q435.1', 'Q332.1', 'Q90.1', 'Q199.1', 'Q134.1', 'Q340.1', 'Q160.1', 'Q82.1', 'Q355.1', 'Q400.1']
Unique company IDs found for Task 1: ['1', '104', '134', '138', '160', '183', '199', '20', '201', '205', '211', '287', '288', '290', '332', '340', '355', '400', '425', '426', '435', '78', '82', '90']


In [ ]:
task2_risk_cols = [col for col in survey.columns if re.match(r'Q\d+\.2_\d+$', col)]

### Krippendorff's alpha

It's an inter-rater agreement that could handle missing values and multiple annotators. Since the survey is multiple choice, each risk domain for each company will be columns and each annotator as rows. It will have values 1 if the annotator selected that domain for that company or 0 if they didnt and Nan if the annotator did not have that company in their survey. It was calculated on the final output of task 2 since task 2 receives the outputs of task 1 and to not complicate things further, I just calculated all together on task 2.

In [ ]:
all_annotators_combined_task2_data = []

# Iterate through each annotator
for annotator_id in unique_annotator_ids:
    # Get all rows for the current annotator from the survey DataFrame
    annotator_data = survey[survey['Dem_1'] == annotator_id]

    # Check if data exists for this annotator
    if annotator_data.empty:
        # If no data, append a row of NaNs for this annotator for all possible columns
        all_annotators_combined_task2_data.append([np.nan] * (len(all_task1_company_ids) * len(risk_categories)))
        continue


    # Initialize a list to hold this annotator's Task 2 ratings across all companies and risks
    annotator_ratings_across_companies_task2 = []

    # Iterate through all unique company IDs
    for company_id_str in all_task1_company_ids:
        company_task2_risk_cols = [col for col in task2_risk_cols if col.startswith(f'Q{company_id_str}.2_')]
        company_task2_risk_cols = sorted(company_task2_risk_cols, key=lambda x: int(re.search(r'_(\d+)$', x).group(1)))
        annotator_company_ratings_task2 = {risk: np.nan for risk in risk_categories}
        annotator_survey_id = annotator_data.iloc[0]['survey_id']
        companies_answered_by_annotator = survey_companies_mapping.get(annotator_survey_id, [])


        if company_id_str in companies_answered_by_annotator:
            # Select Task 2 columns for this specific annotator and company
            annotator_task2_data_raw = annotator_data[company_task2_risk_cols]

            # Convert 'Yes' to 1, 'No' to 0, and others to NaN. NaN means that annotator didnt select that risks in task 1 or did not assesed the company
            annotator_task2_data_numeric = annotator_task2_data_raw.apply(
                lambda series: series.map(
                    lambda x: 1.0 if str(x).strip().lower() == 'yes' else \
                              (0.0 if str(x).strip().lower() == 'no' else np.nan)
                )
            )


            # Iterate through the numeric Task 2 data for this annotator and company
            for col_name, rating_series in annotator_task2_data_numeric.items():
                 # Extract the risk numeric ID from the column name
                 match = re.search(r'Q\d+\.2_(\d+)$', col_name)
                 if match:
                     survey_risk_id = int(match.group(1))

                     # Use the Task 2 mapping to get the correct risk category name
                     if survey_risk_id in survey_risk_id_to_category_index_task2:
                         category_index = survey_risk_id_to_category_index_task2[survey_risk_id]
                         risk_name = risk_categories[category_index]

                         # Update the rating for this risk for the annotator
                         annotator_company_ratings_task2[risk_name] = rating_series.iloc[0] # Get the single value from the Series
        # Append the ratings for this company (list of 0s, 1s, or NaNs for each risk)
        annotator_ratings_across_companies_task2.extend(list(annotator_company_ratings_task2.values()))

    # Append the combined Task 2 ratings for this annotator across all companies and risks
    all_annotators_combined_task2_data.append(annotator_ratings_across_companies_task2)


# Convert the combined Task 2 data to a NumPy array
combined_overall_data_task2 = np.array(all_annotators_combined_task2_data)

unique_values_task2_overall = np.unique(combined_overall_data_task2[~np.isnan(combined_overall_data_task2)])

if len(unique_values_task2_overall) > 1:
     # Calculate overall alpha for Task 2
     alpha_overall_task2 = krippendorff.alpha(reliability_data=combined_overall_data_task2.T, level_of_measurement='nominal') # Transpose for krippendorff
     print(f"\nOverall Krippendorff's Alpha for all data (Task 2): {alpha_overall_task2:.4f}")


Overall Krippendorff's Alpha for all data (Task 2): 0.1741


**Processing**

To extract the risks by majority voting, the open text responses from the majority voting and the risks that conflict with the mission by majority voting alongside their explanations of only those majority voted domains. This will be merged with the dataset of companies, companies_survey.

In [ ]:
#risks categories by majority voting
from collections import Counter

def get_majority_vote_list_task1(labels_list_from_column):
    counter = Counter()
    for raw_label_string in labels_list_from_column:
        if pd.notna(raw_label_string):
            # Split by '.,' and take the part before the colon for each label
            labels = [label.split(':')[0].strip() for label in str(raw_label_string).split('.,')]
            counter.update(labels)
    # Return labels selected by at least 2 annotators as a list
    majority_voted_labels = [label for label, count in counter.items() if count >= 2]
    return majority_voted_labels

In [ ]:
task1_majority_by_company = {}

for q_column_name in company_questions_task1:
    match = re.match(r'Q(\d+)', q_column_name)
    if match:
        company_id_str = match.group(1)
        raw_annotations_for_company = survey[q_column_name].tolist()
        majority_risks_list_task1 = get_majority_vote_list_task1(raw_annotations_for_company)

        task1_majority_by_company[company_id_str] = majority_risks_list_task1

In [ ]:
task1_majority_df = pd.DataFrame(
    list(task1_majority_by_company.items()),
    columns=['ID', 'Task1_majority_risk_survey']
)

# Convert the 'ID' column in task1_majority_df to string type to match companies_survey['Company ID']
task1_majority_df['ID'] = task1_majority_df['ID'].astype(str)

#companies_survey is the base DataFrame
final_companies_survey = pd.merge(
    companies_survey,
    task1_majority_df,
    left_on='Company ID',
    right_on='ID',
    how='left'
)

final_companies_survey = final_companies_survey.drop('ID_y', axis=1)
final_companies_survey = final_companies_survey.rename(columns={'ID_x': 'ID'}) # Rename the original ID column back


final_companies_survey

,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Sector BvD,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,Company ID,Task1_majority_risk_survey
0,201,Korea Electric Power,"$66,977.2",0.226,"$-3,692.4",-,"$185,048.1",48696,272,KR,...,NaN,- The Company was established for the purpose ...,752,"AI-assisted power grid optimization, AI for pr...","Predictive Grid Failure Detection, AI-driven E...","AI for predictive maintenance of generators, O...","AI for predictive maintenance of generators, O...",Korea Electric Power generates and sells elect...,201,"[Malicious actors, Socioeconomic & Environment..."
1,104,Société Générale,"$99,163.4",0.564,"$2,695.1",0.27,"$1,716,418.2",124089,978,FR,...,"Banking, Insurance & Financial Services",Société Générale is a major French multination...,284,"Fraud detection in transactions, AI-based fina...","AI-powered credit risk assessment, Chatbots fo...","AI for fraud detection, Personalized financial...","AI for fraud detection, Personalized financial...",Société Générale provides diverse financial se...,104,"[Discrimination & Toxicity, Misinformation, Pr..."
2,183,Airbus,"$70,751",0.145,"$4,096.1",-0.083,"$131,291.1",147893,250,NL,...,Transport Manufacturing,The Company is engaged in manufacturing aircra...,2302,"AI-powered predictive maintenance, Autonomous ...","AI predicts aircraft maintenance schedules, En...","AI for predictive maintenance of aircraft, AI ...","AI predicts aircraft maintenance schedules, En...","Airbus designs, manufactures, and sells commer...",183,"[Malicious actors, Human- Computer Interaction..."
3,205,Deutsche Bank,"$65,978.1",0.56,"$6,845.3",0.201,"$1,454,899.5",90130,828,DE,...,"Banking, Insurance & Financial Services",Deutsche Bank AG is a global financial service...,396,"AI-driven portfolio management, Fraud detectio...","AI-enhanced credit risk assessment, Chatbots f...","AI for fraud detection, AI-driven trading algo...","AI-enhanced credit risk assessment, Chatbots f...",Deutsche Bank is a global financial services p...,205,"[Discrimination & Toxicity, Misinformation, Pr..."
4,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,Retail,"Walmart Inc., incorporated on October 31, 1969...",3124,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,1,"[Misinformation, Human- Computer Interaction, ..."
5,425,Olam Group,"$35,952.8",-0.098,$207.6,-0.545,"$25,287.1",65980,663,SG,...,NaN,Olam Group Limited (the 'Company') is a limite...,1738,"Optimize crop yield predictions, AI-powered su...",AI forecasts crop yields for supplier optimiza...,"AI for crop yield prediction, AI-driven invent...","AI for crop yield prediction, AI-driven invent...",Olam Group is a leading food and agribusiness ...,425,"[Misinformation, Privacy & Security, Malicious..."
6,78,China Southern Power Grid,"$118,813.5",0.045,"$2,342.2",0.546,"$173,039.2",268471,2780,CN,...,NaN,Construction and management of the southern re...,489,"AI predictive maintenance for transformers, AI...","Predictive Maintenance for Grid Equipment, AI-...","AI for predictive maintenance of power lines, ...","Predictive Maintenance for Grid Equipment, AI-...",Operates and manages southern China's regional...,78,"[Privacy & Security, Malicious actors, Human- ..."
7,211,Accenture,"$64,111.8",0.041,"$6,871.6",-0.001,"$51,245.3",732819,298,IE,...,Business Services,The Company is a global professional services ...,2028,"AI for autonomous network optimization, AI-pow...","AI-driven project risk analysis, Chatbots for ...","AI-driven project risk management, Personalize...","AI-driven project risk analysis, Chatbots for ...",Accenture provides digital transformation and ...,211,"[Discrimination & Toxicity, Misinform

In [ ]:
# Calculate majority-voted risks for Task 2

majority_risks_by_company = {}

for company_id_str in company_ids_task2:
    # Get all Task 2 risk columns specific to the current company
    company_task2_risk_cols = [col for col in task2_columns if col.startswith(f'Q{company_id_str}.2_')]

    if not company_task2_risk_cols:
        continue

    company_data_raw = survey[company_task2_risk_cols]

    # Convert 'Yes' to 1, 'No' to 0, and all other values (blanks, unexpected text) to NaN
    # Using .apply(lambda series: series.map(...)) as recommended by the FutureWarning
    reliability_data_numeric = company_data_raw.apply(
        lambda series: series.map(
            lambda x: 1 if str(x).strip().lower() == 'yes' else \
                      (0 if str(x).strip().lower() == 'no' else np.nan)
        )
    ).astype(float) # Ensure float type to correctly handle np.nan

    current_company_majority_risks = []

    # Iterate through each risk category (column) for the current company
    for col_name, column_data in reliability_data_numeric.items():
        # Extract the risk category number (e.g., from 'Q1.2_5', get '5')
        match = re.search(r'_(\d+)$', col_name)
        if match:
            risk_numeric_id = int(match.group(1)) # Get the number (1 to 24)
            risk_idx = risk_numeric_id - 1       # Convert to 0-indexed for `risk_categories` list

            # Ensure the extracted index is valid for your risk_categories list
            if 0 <= risk_idx < len(risk_categories):
                risk_name = risk_categories[risk_idx]

                # Count 'Yes' votes (which are 1s in our numeric data)
                yes_votes = column_data.sum()

                # Check if at least 2 annotators marked 'Yes'
                if yes_votes >= 2:
                    current_company_majority_risks.append(risk_name)

    # Store the list of majority-voted risks for the current company ID
    majority_risks_by_company[company_id_str] = current_company_majority_risks

# Create a DataFrame from the majority-voted risks for Task 2
majority_risks_df = pd.DataFrame(
    list(majority_risks_by_company.items()),
    columns=['Company ID', 'Task2_majority_risk_survey'])

In [ ]:
final_companies_survey = pd.merge(
    final_companies_survey,
    majority_risks_df,
    on='Company ID', # Change from 'ID' to 'Company ID' to match majority_risks_df
    how='left'
)

In [ ]:
final_companies_survey

,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Description,l,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,Company ID,Task1_majority_risk_survey,Task2_majority_risk_survey
0,201,Korea Electric Power,"$66,977.2",0.226,"$-3,692.4",-,"$185,048.1",48696,272,KR,...,- The Company was established for the purpose ...,752,"AI-assisted power grid optimization, AI for pr...","Predictive Grid Failure Detection, AI-driven E...","AI for predictive maintenance of generators, O...","AI for predictive maintenance of generators, O...",Korea Electric Power generates and sells elect...,201,"[Malicious actors, Socioeconomic & Environment...","[Privacy & Security, Malicious actors, Human- ..."
1,104,Société Générale,"$99,163.4",0.564,"$2,695.1",0.27,"$1,716,418.2",124089,978,FR,...,Société Générale is a major French multination...,284,"Fraud detection in transactions, AI-based fina...","AI-powered credit risk assessment, Chatbots fo...","AI for fraud detection, Personalized financial...","AI for fraud detection, Personalized financial...",Société Générale provides diverse financial se...,104,"[Discrimination & Toxicity, Misinformation, Pr...","[Discrimination & Toxicity, Misinformation, Pr..."
2,183,Airbus,"$70,751",0.145,"$4,096.1",-0.083,"$131,291.1",147893,250,NL,...,The Company is engaged in manufacturing aircra...,2302,"AI-powered predictive maintenance, Autonomous ...","AI predicts aircraft maintenance schedules, En...","AI for predictive maintenance of aircraft, AI ...","AI predicts aircraft maintenance schedules, En...","Airbus designs, manufactures, and sells commer...",183,"[Malicious actors, Human- Computer Interaction...","[Privacy & Security, Malicious actors, Human- ..."
3,205,Deutsche Bank,"$65,978.1",0.56,"$6,845.3",0.201,"$1,454,899.5",90130,828,DE,...,Deutsche Bank AG is a global financial service...,396,"AI-driven portfolio management, Fraud detectio...","AI-enhanced credit risk assessment, Chatbots f...","AI for fraud detection, AI-driven trading algo...","AI-enhanced credit risk assessment, Chatbots f...",Deutsche Bank is a global financial services p...,205,"[Discrimination & Toxicity, Misinformation, Pr...","[Discrimination & Toxicity, Misinformation, Pr..."
4,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,"Walmart Inc., incorporated on October 31, 1969...",3124,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,1,"[Misinformation, Human- Computer Interaction, ...","[Misinformation, AI system safety, failures, &..."
5,425,Olam Group,"$35,952.8",-0.098,$207.6,-0.545,"$25,287.1",65980,663,SG,...,Olam Group Limited (the 'Company') is a limite...,1738,"Optimize crop yield predictions, AI-powered su...",AI forecasts crop yields for supplier optimiza...,"AI for crop yield prediction, AI-driven invent...","AI for crop yield prediction, AI-driven invent...",Olam Group is a leading food and agribusiness ...,425,"[Misinformation, Privacy & Security, Malicious...","[Discrimination & Toxicity, Misinformation, Pr..."
6,78,China Southern Power Grid,"$118,813.5",0.045,"$2,342.2",0.546,"$173,039.2",268471,2780,CN,...,Construction and management of the southern re...,489,"AI predictive maintenance for transformers, AI...","Predictive Maintenance for Grid Equipment, AI-...","AI for predictive maintenance of power lines, ...","Predictive Maintenance for Grid Equipment, AI-...",Operates and manages southern China's regional...,78,"[Privacy & Security, Malicious actors, Human- ...","[Privacy & Security, Malicious actors, Human- ..."
7,211,Accenture,"$64,111.8",0.041,"$6,871.6",-0.001,"$51,245.3",732819,298,IE,...,The Company is a global professional services ...,2028,"AI for autonomous network optimization, AI-pow...","AI-driven pr

The lines below aggregate the text responses for the majority risks for each task into one list per company per task

In [ ]:
# Aggregate Task 1 text responses for risks selected by the majority, by company
aggregated_majority_task1_text_by_company = {}

# Iterate through each company that had Task 1 questions
for company_id_str in all_task1_company_ids:

    # List to store all text responses for majority risks for this company, from all annotators
    all_majority_texts_for_company = []

    # Get the list of majority-voted risks for this company from the final_companies_survey DataFrame
    majority_risks_row = final_companies_survey[final_companies_survey['Company ID'] == company_id_str]

    if majority_risks_row.empty:
        aggregated_majority_task1_text_by_company[company_id_str] = ''
        continue

    # Get the list of majority risks for this company
    majority_risks_for_company = majority_risks_row.iloc[0].get('Task1_majority_risk_survey')

    # Ensure majority_risks_for_company is a list, handle potential NaN or None
    if not isinstance(majority_risks_for_company, list):
        majority_risks_for_company = []

    # Iterate through each annotator (row in the survey DataFrame)
    for index, row_data in survey.iterrows():
        # Iterate through the majority-voted risks for this company
        for risk_name in majority_risks_for_company:
            # Find the corresponding survey_risk_id for this risk_name using the mapping
            survey_risk_id = None
            for id, cat_index in survey_risk_id_to_category_index.items():
                if risk_categories[cat_index] == risk_name:
                    survey_risk_id = id
                    break

            if survey_risk_id is not None:
                text_col_name = f'Q{company_id_str}.1_{survey_risk_id}_TEXT' #the format of text responses
                if text_col_name in row_data.index:
                    raw_text_response = row_data.get(text_col_name, None)
                    # Add the text response if it's not empty and not 'N/A'
                    if pd.notna(raw_text_response) and str(raw_text_response).strip() != '' and str(raw_text_response).strip().lower() != 'n/a':
                        all_majority_texts_for_company.append(str(raw_text_response).strip())


    # Aggregate all collected text responses for the company into a single comma-separated string
    aggregated_majority_task1_text = ', '.join(all_majority_texts_for_company)

    # Store the aggregated text string for the current company
    aggregated_majority_task1_text_by_company[company_id_str] = aggregated_majority_task1_text


# Convert the aggregated data into a DataFrame
data_list = []
for company_id, aggregated_text in aggregated_majority_task1_text_by_company.items():
    data_list.append({
        'Company ID': company_id,
        'Task1_majority_texts_survey': aggregated_text
    })

majority_task1_text_aggregated_df = pd.DataFrame(data_list)

# Display the resulting DataFrame
display(majority_task1_text_aggregated_df)

,Company ID,Task1_majority_texts_survey
0,1,Chatbots can provide vague and unhelpful tips ...
1,104,AIs are trained on media that could be biased ...
2,134,AI systems processing customs clearance expose...
3,138,"If AI doesn't predict maintenance well, the em..."
4,160,AI target recognition may inherit or amplify b...
5,183,The systems could be hacked to provide dangero...
6,199,"AI has no understanding of disability, or risk..."
7,20,Identity theft and financial harm can occur if...
8,201,Malicious actors could hack the system to conv...
9,205,Biased credit algorithms or unequal chatbot pe...


In [ ]:
final_companies_survey = pd.merge(
    final_companies_survey,
    majority_task1_text_aggregated_df,
    on='Company ID',
    how='left'
)

In [ ]:
# Aggregate Task 2 text responses for risks selected by the majority, by company
aggregated_majority_task2_text_by_company = {}

# Iterate through each company that had Task 2 questions
for company_id_str in all_task1_company_ids:

    # List to store all text responses for majority risks for this company (Task 2), from all annotators
    all_majority_texts_for_company_task2 = []

    # Get the list of majority-voted risks for this company from the final_companies_survey DataFrame for Task 2
    majority_risks_row_task2 = final_companies_survey[final_companies_survey['Company ID'] == company_id_str]

    if majority_risks_row_task2.empty:
        # or if no majority Task 2 risks were identified, add an empty string and continue.
        aggregated_majority_task2_text_by_company[company_id_str] = ''
        continue

    # Get the list of majority risks for this company for Task 2
    majority_risks_for_company_task2 = majority_risks_row_task2.iloc[0].get('Task2_majority_risk_survey')

    # Ensure majority_risks_for_company_task2 is a list, handle potential NaN or None
    if not isinstance(majority_risks_for_company_task2, list):
        majority_risks_for_company_task2 = []

    # Iterate through each annotator (row in the survey DataFrame)
    for index, row_data in survey.iterrows():
        # Iterate through the majority-voted risks for this company for Task 2
        for risk_name in majority_risks_for_company_task2:
            # Find the corresponding survey_risk_id for this risk_name using the Task 2 mapping
            survey_risk_id_task2 = None
            for id, cat_index in survey_risk_id_to_category_index_task2.items():
                if risk_categories[cat_index] == risk_name:
                    survey_risk_id_task2 = id
                    break

            if survey_risk_id_task2 is not None:
                # Construct the original column name for the Task 2 text response
                text_col_name_task2 = f'Q{company_id_str}.2_{survey_risk_id_task2}_TEXT'

                # Check if the text column exists for this company and annotator
                if text_col_name_task2 in row_data.index:
                    raw_text_response_task2 = row_data.get(text_col_name_task2, None)
                    # Add the text response if it's not empty and not 'N/A'
                    if pd.notna(raw_text_response_task2) and str(raw_text_response_task2).strip() != '' and str(raw_text_response_task2).strip().lower() != 'n/a':
                        all_majority_texts_for_company_task2.append(str(raw_text_response_task2).strip())


    # Aggregate all collected text responses for the company into a single comma-separated string
    aggregated_majority_task2_text = ', '.join(all_majority_texts_for_company_task2)

    # Store the aggregated text string for the current company
    aggregated_majority_task2_text_by_company[company_id_str] = aggregated_majority_task2_text


# Convert the aggregated data into a DataFrame
data_list_task2 = []
for company_id, aggregated_text in aggregated_majority_task2_text_by_company.items():
    data_list_task2.append({
        'Company ID': company_id,
        'Task2_majority_texts_survey': aggregated_text
    })

majority_task2_text_aggregated_df = pd.DataFrame(data_list_task2)

# Display the resulting DataFrame
display(majority_task2_text_aggregated_df)

,Company ID,Task2_majority_texts_survey
0,1,It would only cause frustrations for customers...
1,104,"It conflicts with building ""together"" since it..."
2,134,Breaches of personal or commercial data compro...
3,138,If there's no human control over AI and it doe...
4,160,Compromised target recognition undermines nati...
5,183,"It conflicts with keeping a ""safe"" world., It ..."
6,199,"Failing to account for disabilities, race, gen..."
7,20,
8,201,This conflicts with their commitment to a brig...
9,205,Unfair treatment through biased AI blocks acce...


In [ ]:
final_companies_survey = pd.merge(
    final_companies_survey,
    majority_task2_text_aggregated_df,
    on='Company ID',
    how='left'
)
#The final output is the survey answers with the majority voted columns for comparison with the LLMs in the third notebook
#final_companies_survey.to_csv("2_companies_survey.csv", index=False)

In [ ]:
final_companies_survey

,ID,Company,Revenues ($M),Revenue Percent Change,Profits ($M),Profits Percent Change,Assets ($M),Employees,ID BvD,ISO,...,Usecases_1,Usecases_2,Usecases_3,Usecases_USC,Description_summary,Company ID,Task1_majority_risk_survey,Task2_majority_risk_survey,Task1_majority_texts_survey,Task2_majority_texts_survey
0,201,Korea Electric Power,"$66,977.2",0.226,"$-3,692.4",-,"$185,048.1",48696,272,KR,...,"AI-assisted power grid optimization, AI for pr...","Predictive Grid Failure Detection, AI-driven E...","AI for predictive maintenance of generators, O...","AI for predictive maintenance of generators, O...",Korea Electric Power generates and sells elect...,201,"[Malicious actors, Socioeconomic & Environment...","[Privacy & Security, Malicious actors, Human- ...",Malicious actors could hack the system to conv...,This conflicts with their commitment to a brig...
1,104,Société Générale,"$99,163.4",0.564,"$2,695.1",0.27,"$1,716,418.2",124089,978,FR,...,"Fraud detection in transactions, AI-based fina...","AI-powered credit risk assessment, Chatbots fo...","AI for fraud detection, Personalized financial...","AI for fraud detection, Personalized financial...",Société Générale provides diverse financial se...,104,"[Discrimination & Toxicity, Misinformation, Pr...","[Discrimination & Toxicity, Misinformation, Pr...",AIs are trained on media that could be biased ...,"It conflicts with building ""together"" since it..."
2,183,Airbus,"$70,751",0.145,"$4,096.1",-0.083,"$131,291.1",147893,250,NL,...,"AI-powered predictive maintenance, Autonomous ...","AI predicts aircraft maintenance schedules, En...","AI for predictive maintenance of aircraft, AI ...","AI predicts aircraft maintenance schedules, En...","Airbus designs, manufactures, and sells commer...",183,"[Malicious actors, Human- Computer Interaction...","[Privacy & Security, Malicious actors, Human- ...",The systems could be hacked to provide dangero...,"It conflicts with keeping a ""safe"" world., It ..."
3,205,Deutsche Bank,"$65,978.1",0.56,"$6,845.3",0.201,"$1,454,899.5",90130,828,DE,...,"AI-driven portfolio management, Fraud detectio...","AI-enhanced credit risk assessment, Chatbots f...","AI for fraud detection, AI-driven trading algo...","AI-enhanced credit risk assessment, Chatbots f...",Deutsche Bank is a global financial services p...,205,"[Discrimination & Toxicity, Misinformation, Pr...","[Discrimination & Toxicity, Misinformation, Pr...",Biased credit algorithms or unequal chatbot pe...,Unfair treatment through biased AI blocks acce...
4,1,Walmart,"$648,125",0.06,"$15,511",0.328,"$252,399",2100000,1,US,...,"AI for demand forecasting, AI-enhanced supply ...",AI-driven demand forecasting for inventory man...,"AI-driven dynamic pricing, Predictive inventor...",AI-driven demand forecasting for inventory man...,Walmart is a technology-powered omnichannel re...,1,"[Misinformation, Human- Computer Interaction, ...","[Misinformation, AI system safety, failures, &...",Chatbots can provide vague and unhelpful tips ...,It would only cause frustrations for customers...
5,425,Olam Group,"$35,952.8",-0.098,$207.6,-0.545,"$25,287.1",65980,663,SG,...,"Optimize crop yield predictions, AI-powered su...",AI forecasts crop yields for supplier optimiza...,"AI for crop yield prediction, AI-driven invent...","AI for crop yield prediction, AI-driven invent...",Olam Group is a leading food and agribusiness ...,425,"[Misinformation, Privacy & Security, Malicious...","[Discrimination & Toxicity, Misinformation, Pr...","Inaccurate AI outputs in crop forecasting, inv...",Misleading AI outputs can lead to poor decisio...
6,78,China Southern Power Grid,"$118,813.5",0.045,"$2,342.2",0.546,"$173,039.2",268471,2780,CN,...,"AI predictive maintenance for transformers, AI...","Predictive Maintenance for Grid Equipment, AI-...","AI for predictive maintenance of power lines, ...","Predictive Maintenance for Grid Equipment, AI-...",Operates and manages southern China's regional...,78,"[Privacy & Security, Malicious actor